![Alt Text](img/box-office.avif)

## Predicción de recaudación y beneficio de películas en cartelera

- Extraído de https://www.kaggle.com/competitions/tmdb-box-office-prediction/data


#### Problema: mediante una interfaz, poder intentar predecir, siendo una productora, que género, con que actores, director, duración aproximada o mes de estreno serían los idoneos para un proyecto con x presupuesto

In [1798]:
import pandas as pd
import numpy as np
from bootcampviztools import *
from Toolbox_ML import *
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from collections import defaultdict, OrderedDict
import re
import pickle

In [1803]:
df = pd.read_csv('train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

- Columnas que importan: id, budget, genres, original_language, title, production_countries, release_date, runtime, spoken_languages, cast, crew, REVENUE

In [ ]:
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [ ]:
df = df.drop(['belongs_to_collection', 'spoken_languages', 'homepage', 'Keywords', 'imdb_id', 'overview','popularity','poster_path', 'status','tagline','original_title'], axis=1)

In [ ]:
df.head()

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue
0,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",en,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,Hot Tub Time Machine 2,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,The Princess Diaries 2: Royal Engagement,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,3300000,"[{'id': 18, 'name': 'Drama'}]",en,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,Whiplash,"[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",hi,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,Kahaani,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",ko,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,Marine Boy,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


- Muestra de géneros:

In [ ]:
df.genres[0]

"[{'id': 35, 'name': 'Comedy'}]"

In [ ]:
df.genres[17]

"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"

- Muestra de Cast:

In [ ]:
df.cast[0]

"[{'cast_id': 4, 'character': 'Lou', 'credit_id': '52fe4ee7c3a36847f82afae7', 'gender': 2, 'id': 52997, 'name': 'Rob Corddry', 'order': 0, 'profile_path': '/k2zJL0V1nEZuFT08xUdOd3ucfXz.jpg'}, {'cast_id': 5, 'character': 'Nick', 'credit_id': '52fe4ee7c3a36847f82afaeb', 'gender': 2, 'id': 64342, 'name': 'Craig Robinson', 'order': 1, 'profile_path': '/tVaRMkJXOEVhYxtnnFuhqW0Rjzz.jpg'}, {'cast_id': 6, 'character': 'Jacob', 'credit_id': '52fe4ee7c3a36847f82afaef', 'gender': 2, 'id': 54729, 'name': 'Clark Duke', 'order': 2, 'profile_path': '/oNzK0umwm5Wn0wyEbOy6TVJCSBn.jpg'}, {'cast_id': 7, 'character': 'Adam Jr.', 'credit_id': '52fe4ee7c3a36847f82afaf3', 'gender': 2, 'id': 36801, 'name': 'Adam Scott', 'order': 3, 'profile_path': '/5gb65xz8bzd42yjMAl4zwo4cvKw.jpg'}, {'cast_id': 8, 'character': 'Hot Tub Repairman', 'credit_id': '52fe4ee7c3a36847f82afaf7', 'gender': 2, 'id': 54812, 'name': 'Chevy Chase', 'order': 4, 'profile_path': '/svjpyYtPwtjvRxX9IZnOmOkhDOt.jpg'}, {'cast_id': 9, 'character

- Muestra de Crew:

In [ ]:
df.crew[0]

'[{\'credit_id\': \'59ac067c92514107af02c8c8\', \'department\': \'Directing\', \'gender\': 0, \'id\': 1449071, \'job\': \'First Assistant Director\', \'name\': \'Kelly Cantley\', \'profile_path\': None}, {\'credit_id\': \'52fe4ee7c3a36847f82afad7\', \'department\': \'Directing\', \'gender\': 2, \'id\': 3227, \'job\': \'Director\', \'name\': \'Steve Pink\', \'profile_path\': \'/myHOgo8mQSCiCAZNGMRdHVr03jr.jpg\'}, {\'credit_id\': \'5524ed25c3a3687ded000d88\', \'department\': \'Writing\', \'gender\': 2, \'id\': 347335, \'job\': \'Writer\', \'name\': \'Josh Heald\', \'profile_path\': \'/pwXJIenrDMrG7t3zNfLvr8w1RGU.jpg\'}, {\'credit_id\': \'5524ed2d925141720c001128\', \'department\': \'Writing\', \'gender\': 2, \'id\': 347335, \'job\': \'Characters\', \'name\': \'Josh Heald\', \'profile_path\': \'/pwXJIenrDMrG7t3zNfLvr8w1RGU.jpg\'}, {\'credit_id\': \'5524ed3d92514166c1004a5d\', \'department\': \'Production\', \'gender\': 2, \'id\': 57822, \'job\': \'Producer\', \'name\': \'Andrew Panay\', \

## Cleaning

#### Budget and Revenue incongruences

- We may find in this database loads of movies with budget 0 (whic is not possible, every movie has a budget), really lows budgets or really low revenues. We will apply some transformations to modify this

In [ ]:
print("We have", len(df[df.budget == 0]), "with budget = 0")

We have 812 with budget = 0


In [ ]:
df = df[df.budget != 0]

In [ ]:
df[(df.budget > 0) & (df.budget < 10000)].head()

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue
334,335,1,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",ta,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",5/5/03,140.0,Saamy,"[{'cast_id': 11, 'character': 'Aaruchamy', 'cr...","[{'credit_id': '52fe4782c3a36847f813999f', 'de...",3
347,348,12,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",en,"[{'name': 'Allied Filmmakers', 'id': 1755}]","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",10/16/96,84.0,The Wind in the Willows,"[{'cast_id': 2, 'character': 'Toad', 'credit_i...","[{'credit_id': '59ce7655c3a3686aca01a522', 'de...",1
365,366,7000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",en,"[{'name': 'Thinkfilm', 'id': 446}]","[{'iso_3166_1': 'US', 'name': 'United States o...",10/8/04,77.0,Primer,"[{'cast_id': 1, 'character': 'Aaron', 'credit_...","[{'credit_id': '52fe45e79251416c75066791', 'de...",424760
469,470,113,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",en,"[{'name': 'Columbia Pictures Corporation', 'id...","[{'iso_3166_1': 'US', 'name': 'United States o...",6/18/86,113.0,"The Karate Kid, Part II","[{'cast_id': 1, 'character': 'Daniel LaRusso',...","[{'credit_id': '52fe44c1c3a36847f80a82dd', 'de...",115103979
512,513,93,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'name': 'Lewis Gilbert Productions', 'id': 1...","[{'iso_3166_1': 'US', 'name': 'United States o...",1/28/11,107.0,From Prada to Nada,"[{'cast_id': 9, 'character': 'Nora', 'credit_i...","[{'credit_id': '52fe47bfc3a36847f8146211', 'de...",2500000


- Inspecting these last movies with a budget, we appreciate, searching on the Internet, that the numbers below 12 should be millions, the numbers in hundreds or almost 100 should be thousands, and we drop the rest for the sake of the prediction model, as they could be real cases as Following, or very amateur movies that do not follow the scope of these project

In [ ]:
df['budget'] = np.where(df['budget'] < 13, df['budget'] * 1000000, df['budget'])
df['budget'] = np.where(df['budget'] < 1000, df['budget'] * 1000, df['budget'])
df = df.loc[~((df['budget'] > 0) & (df['budget'] < 10000))]

- We inspect now the movies under 1M$

In [ ]:
df[df.budget < 1000000].head(10)

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue
15,16,500000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",sr,"[{'name': 'Hypnopolis', 'id': 51316}]","[{'iso_3166_1': 'RS', 'name': 'Serbia'}]",10/6/10,97.0,Skinning,"[{'cast_id': 1001, 'character': 'Novica', 'cre...","[{'credit_id': '52fe4927c3a36847f818bb5f', 'de...",198
50,51,560000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",nl,NaN,"[{'iso_3166_1': 'BE', 'name': 'Belgium'}]",1/30/08,102.0,"Moscow, Belgium","[{'cast_id': 3, 'character': 'Matty', 'credit_...","[{'credit_id': '52fe45879251416c75059975', 'de...",1500000
74,75,445000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",en,"[{'name': 'New World Pictures', 'id': 1950}]","[{'iso_3166_1': 'US', 'name': 'United States o...",3/31/76,84.0,Jackson County Jail,"[{'cast_id': 1, 'character': 'Dinah Hunter', '...","[{'credit_id': '52fe442ec3a368484e0141ef', 'de...",2300000
76,77,500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",ta,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",5/5/00,157.0,Kandukondain Kandukondain,"[{'cast_id': 1, 'character': 'Manohar', 'credi...","[{'credit_id': '52fe47c7c3a368484e0d8f41', 'de...",500000
162,163,125000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",en,"[{'name': 'Ada Films', 'id': 1424}]","[{'iso_3166_1': 'US', 'name': 'United States o...",9/11/11,90.0,Your Sister's Sister,"[{'cast_id': 2, 'character': 'Jack', 'credit_i...","[{'credit_id': '53b6f5880e0a2676b8005b39', 'de...",3200000
187,188,259000,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",en,"[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8...","[{'iso_3166_1': 'US', 'name': 'United States o...",11/24/28,65.0,West of Zanzibar,"[{'cast_id': 9, 'character': ""Phroso 'Dead-Leg...","[{'credit_id': '52fe454fc3a368484e052d29', 'de...",921000
231,232,800000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,"[{'name': 'Ruby in Paradise', 'id': 75712}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/8/93,114.0,Ruby in Paradise,"[{'cast_id': 3, 'character': 'Ruby Lee Gissing...","[{'credit_id': '52fe474cc3a36847f812dcf7', 'de...",1001437
254,255,11178,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'name': 'APT Entertainment', 'id': 8355}, {'...","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",9/5/12,104.0,Voracious,"[{'cast_id': 16, 'character': 'Rene', 'credit_...","[{'credit_id': '52fe4a5cc3a368484e14eef3', 'de...",34659
259,260,24554,"[{'id': 18, 'name': 'Drama'}]",en,NaN,NaN,9/12/98,95.0,The Terrorist,"[{'cast_id': 2, 'character': 'Malli', 'credit_...","[{'credit_id': '52fe475bc3a36847f813138f', 'de...",140021
267,268,27000,"[{'id': 35, 'name': 'Comedy'}]",en,"[{'name': 'Miramax Films', 'id': 14}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",9/13/94,92.0,Clerks,"[{'cast_id': 7, 'character': 'Dante Hicks', 'c...","[{'credit_id': '52fe434ac3a36847f8049259', 'de...",3151130


In [ ]:
df = df[df.title != "THX 1138"]
df = df[df.revenue != "The Girlfriend Experience"]
df = df[df.revenue != "Pi"]
df = df[df.revenue != "Deep Rising"]
df = df[df.revenue != "Christopher Columbus: The Discovery"]
df = df[df.revenue != "Running with Scissorse"]
df = df[df.revenue != "The Frighteners"]
df = df[df.revenue != "Bitter Moon"]
df = df[df.revenue != "Connie and Carla"]

- They seem fine, budgets according to the real ones.

In [ ]:
print("We have", len(df[df.revenue < 1000]), "with revenue < 1000")

We have 23 with revenue < 1000


In [ ]:
df[df.revenue < 100].head(10)

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue
312,313,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'name': 'Cookout Productions', 'id': 3494}]","[{'iso_3166_1': 'US', 'name': 'United States o...",9/3/04,97.0,The Cookout,"[{'cast_id': 1, 'character': 'blin bling', 'cr...","[{'credit_id': '52fe477f9251416c7509b8ed', 'de...",12
334,335,1000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",ta,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",5/5/03,140.0,Saamy,"[{'cast_id': 11, 'character': 'Aaruchamy', 'cr...","[{'credit_id': '52fe4782c3a36847f813999f', 'de...",3
347,348,12000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",en,"[{'name': 'Allied Filmmakers', 'id': 1755}]","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",10/16/96,84.0,The Wind in the Willows,"[{'cast_id': 2, 'character': 'Toad', 'credit_i...","[{'credit_id': '59ce7655c3a3686aca01a522', 'de...",1
450,451,23000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",en,"[{'name': 'Alcon Entertainment', 'id': 1088}, ...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1/9/04,111.0,Chasing Liberty,"[{'cast_id': 1, 'character': 'Anna Foster', 'c...","[{'credit_id': '563c813992514150af00414f', 'de...",12
639,640,6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,"[{'name': 'Fred Berner Films', 'id': 89312}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",9/6/00,122.0,Pollock,"[{'cast_id': 1, 'character': 'Jackson Pollock'...","[{'credit_id': '52fe44f19251416c75046447', 'de...",8
695,696,1000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 53, ...",hi,"[{'name': 'People Tree Films', 'id': 11791}]","[{'iso_3166_1': 'IN', 'name': 'India'}]",8/15/03,132.0,Tere Naam,"[{'cast_id': 3, 'character': 'Radhe Mohan', 'c...","[{'credit_id': '52fe491ec3a36847f8189cfb', 'de...",2
1006,1007,2000000,"[{'id': 53, 'name': 'Thriller'}]",en,"[{'name': 'Zyzzyx LLC', 'id': 92534}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2/25/06,90.0,Zyzzyx Road,"[{'cast_id': 1, 'character': 'Marissa', 'credi...","[{'credit_id': '52fe45c6c3a36847f80d9bd3', 'de...",30
1198,1199,5000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",en,"[{'name': 'Malpaso Productions', 'id': 171}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",12/19/78,110.0,Every Which Way But Loose,"[{'cast_id': 1, 'character': 'Philo Beddoe', '...","[{'credit_id': '52fe46579251416c75075487', 'de...",85
1281,1282,9000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'name': 'Sidney Kimmel Entertainment', 'id':...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",2/9/07,90.0,Death at a Funeral,"[{'cast_id': 4, 'character': 'Daniel', 'credit...","[{'credit_id': '52fe4340c3a36847f8045d0d', 'de...",46
1346,1347,1000000,"[{'id': 18, 'name': 'Drama'}]",en,"[{'name': 'Warner Bros.', 'id': 6194}]","[{'iso_3166_1': 'US', 'name': 'United States o...",3/9/55,115.0,East of Eden,"[{'cast_id': 4, 'character': 'Caleb Trask', 'c...","[{'credit_id': '52fe4228c3a36847f800869d', 'de...",5


- We erase Zyzzyx Road due to being the worst release ever, a total flop due to release problems. I could not also find the real box office for A Farewell to Arms (1932)

In [ ]:
df = df.loc[~((df['title'] == 'A Farewell to Arms') | (df['title'] == 'Zyzzyx Road'))]

- We find the same issue as with budget, the movies below 25 should be millons

In [ ]:
df['revenue'] = np.where(df['revenue'] < 100, df['revenue'] * 1000000, df['revenue'])

In [ ]:
df[df.revenue < 1000].head(10)

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue
15,16,500000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",sr,"[{'name': 'Hypnopolis', 'id': 51316}]","[{'iso_3166_1': 'RS', 'name': 'Serbia'}]",10/6/10,97.0,Skinning,"[{'cast_id': 1001, 'character': 'Novica', 'cre...","[{'credit_id': '52fe4927c3a36847f818bb5f', 'de...",198
1800,1801,5000000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",en,NaN,"[{'iso_3166_1': 'GR', 'name': 'Greece'}, {'iso...",1/1/99,141.0,The Cherry Orchard,[],"[{'credit_id': '52fe471bc3a36847f8123c13', 'de...",135
2255,2256,1000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",en,"[{'name': 'Alcon Entertainment', 'id': 1088}, ...","[{'iso_3166_1': 'US', 'name': 'United States o...",4/23/99,95.0,Lost & Found,"[{'cast_id': 1, 'character': 'Dylan Ramsey', '...","[{'credit_id': '52fe47429251416c9106f659', 'de...",100
2864,2865,410000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",de,"[{'name': 'Österreichischer Rundfunk (ORF)', '...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",2/19/72,101.0,The Goalie's Anxiety at the Penalty Kick,"[{'cast_id': 4, 'character': 'Josef Bloch', 'c...","[{'credit_id': '5925158bc3a3687870008efc', 'de...",100


- We also do not know the revenue for these movies

In [ ]:
df = df.loc[~((df['revenue'] < 1000))]

In [ ]:
df[df.revenue < 10000].head(10)

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue
96,97,2500000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",fr,"[{'name': 'Procirep', 'id': 311}, {'name': 'Ca...","[{'iso_3166_1': 'FR', 'name': 'France'}]",11/14/11,98.0,Sleepless Night,"[{'cast_id': 1, 'character': 'Vincent', 'credi...","[{'credit_id': '52fe49c0c3a368484e13dc9b', 'de...",3358
131,132,6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",en,NaN,"[{'iso_3166_1': 'US', 'name': 'United States o...",1/1/11,93.0,The Good Doctor,"[{'cast_id': 3, 'character': 'Dr. Martin Blake...","[{'credit_id': '52fe4714c3a368484e0b3c91', 'de...",5206
494,495,2000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",en,"[{'name': 'Capitol Films', 'id': 826}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/09,97.0,Stolen,"[{'cast_id': 2, 'character': 'Tom Adkins Sr.',...","[{'credit_id': '52fe46e69251416c910637ad', 'de...",7306
820,821,4000000,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",en,"[{'name': 'Worldview Entertainment', 'id': 901...","[{'iso_3166_1': 'US', 'name': 'United States o...",9/2/13,99.0,The Sacrament,"[{'cast_id': 4, 'character': 'Jake', 'credit_i...","[{'credit_id': '536cd3390e0a2647db00d5d1', 'de...",9221
938,939,1500000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'name': 'KODA Entertainment', 'id': 16947}]","[{'iso_3166_1': 'US', 'name': 'United States o...",10/20/12,90.0,Best Man Down,"[{'cast_id': 2, 'character': 'Scott', 'credit_...","[{'credit_id': '52fe4c42c3a368484e1af3cf', 'de...",1938
1062,1063,1200000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",en,"[{'name': 'Kingsize Entertainment', 'id': 36242}]","[{'iso_3166_1': 'US', 'name': 'United States o...",6/16/00,85.0,Poor White Trash,"[{'cast_id': 2, 'character': 'Sandy Lake', 'cr...","[{'credit_id': '52fe46a2c3a36847f8109a7d', 'de...",1404
1262,1263,5000000,"[{'id': 878, 'name': 'Science Fiction'}]",en,"[{'name': 'Trimark Pictures', 'id': 4063}]","[{'iso_3166_1': 'US', 'name': 'United States o...",11/12/93,97.0,Philadelphia Experiment II,"[{'cast_id': 2, 'character': 'David Herdeg', '...","[{'credit_id': '58ef4d46c3a368264000a993', 'de...",2970
1711,1712,1000000,"[{'id': 27, 'name': 'Horror'}]",en,"[{'name': 'Fewlas Entertainment', 'id': 33606}]","[{'iso_3166_1': 'US', 'name': 'United States o...",9/12/14,87.0,Honeymoon,"[{'cast_id': 1, 'character': 'Bea', 'credit_id...","[{'credit_id': '52fe4eaa9251416c911397f5', 'de...",9318
1755,1756,14000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",en,"[{'name': 'Whiskers Post Production', 'id': 89...","[{'iso_3166_1': 'GR', 'name': 'Greece'}, {'iso...",8/24/07,94.0,Fishtales,"[{'cast_id': 1, 'character': 'Dr. Thomas Bradl...","[{'credit_id': '58fa6367c3a3687a2a000959', 'de...",9216
1977,1978,10000000,"[{'id': 35, 'name': 'Comedy'}]",en,"[{'name': 'Cube Vision', 'id': 2780}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/16/09,85.0,Janky Promoters,"[{'cast_id': 1, 'character': 'Russell Redds', ...","[{'credit_id': '52fe44bcc3a368484e033f5d', 'de...",9069


- For the revenues under 10k$, we find that these are referred to domestic box office only, ignoring the revenue outside the US, and also were flops

In [ ]:
df = df.loc[~((df['revenue'] < 10000))]

In [ ]:
df[df.revenue < 100000].head(10)

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue
39,40,3800000,"[{'id': 10752, 'name': 'War'}, {'id': 37, 'nam...",en,"[{'name': 'Columbia Pictures Corporation', 'id...","[{'iso_3166_1': 'US', 'name': 'United States o...",4/7/65,123.0,Major Dundee,"[{'cast_id': 1, 'character': 'Major Amos Charl...","[{'credit_id': '52fe45f5c3a368484e078d7b', 'de...",14873
222,223,5000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,NaN,"[{'iso_3166_1': 'US', 'name': 'United States o...",6/4/10,98.0,Meet Monica Velour,"[{'cast_id': 1002, 'character': 'Linda Romanol...","[{'credit_id': '5635beffc3a3681b6101a4f8', 'de...",31649
254,255,11178,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'name': 'APT Entertainment', 'id': 8355}, {'...","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",9/5/12,104.0,Voracious,"[{'cast_id': 16, 'character': 'Rene', 'credit_...","[{'credit_id': '52fe4a5cc3a368484e14eef3', 'de...",34659
263,264,10000000,"[{'id': 878, 'name': 'Science Fiction'}]",en,"[{'name': 'Orion Pictures', 'id': 41}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",11/12/87,99.0,Cherry 2000,"[{'cast_id': 1, 'character': ""Edith 'E.' Johns...","[{'credit_id': '52fe467c9251416c75079dad', 'de...",14000
299,300,9000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",en,"[{'name': 'Hemdale Film Corporation', 'id': 469}]","[{'iso_3166_1': 'US', 'name': 'United States o...",11/21/91,94.0,Highway to Hell,"[{'cast_id': 3, 'character': 'Beezle', 'credit...","[{'credit_id': '52fe46b49251416c75080be1', 'de...",26055
311,312,9500000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",fr,"[{'name': 'MK2 Productions', 'id': 9209}]","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",5/18/12,168.0,Laurence Anyways,"[{'cast_id': 1, 'character': 'Laurence Alia', ...","[{'credit_id': '52fe4accc3a36847f81e22eb', 'de...",12250
444,445,10300000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",fr,"[{'name': 'Les Productions du Trésor', 'id': 2...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",10/19/16,120.0,From the Land of the Moon,"[{'cast_id': 0, 'character': 'Gabrielle', 'cre...","[{'credit_id': '568c7e05c3a3684bcc032697', 'de...",47748
473,474,15000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",en,NaN,"[{'iso_3166_1': 'AT', 'name': 'Austria'}, {'is...",10/14/01,99.0,All The Queen's Men,"[{'cast_id': 2, 'character': ""O'Rourke"", 'cred...","[{'credit_id': '52fe44959251416c91015db9', 'de...",23000
492,493,12000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",en,"[{'name': 'StudioCanal', 'id': 694}, {'name': ...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",11/8/12,95.0,Deadfall,"[{'cast_id': 3, 'character': 'Addison', 'credi...","[{'credit_id': '54c8c679c3a36874b70063d8', 'de...",66351
505,506,10000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",fr,"[{'name': 'StudioCanal', 'id': 694}, {'name': ...","[{'iso_3166_1': 'FR', 'name': 'France'}]",11/2/11,109.0,Special Forces,"[{'cast_id': 1003, 'character': 'Elsa', 'credi...","[{'credit_id': '52fe48cec3a368484e10c019', 'de...",10759


- Searching in Box Office Mojo, I substituted the Revenue values from the domestic values to the international ones in the movies which will not be erased for NaN values in the rest of the cleaning, the "B" movies or the disaster flops.

In [ ]:
df.loc[df['title'] == 'Special Forces', 'revenue'] = 3424648
df.loc[df['title'] == 'Deadfall', 'revenue'] = 1946254
df.loc[df['title'] == 'From the Land of the Moon', 'revenue'] = 6547983
df = df.loc[~((df['revenue'] < 100000))]

In [ ]:
df[df.revenue < 1000000].tail(20)

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue
2714,2715,3000000,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",en,[{'name': 'Barclays Mercantile Industrial Fina...,"[{'iso_3166_1': 'US', 'name': 'United States o...",5/29/81,94.0,Dead & Buried,"[{'cast_id': 5, 'character': 'Sheriff Dan Gill...","[{'credit_id': '52fe448ec3a368484e02966d', 'de...",216166
2721,2722,10500000,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",en,"[{'name': 'Nelson Entertainment', 'id': 365}]","[{'iso_3166_1': 'US', 'name': 'United States o...",9/2/94,99.0,There Goes My Baby,"[{'cast_id': 2, 'character': 'Pirate', 'credit...","[{'credit_id': '598db4fcc3a368758e01220c', 'de...",123509
2722,2723,500000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",en,"[{'name': 'Whitewater Films', 'id': 3095}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1/15/04,90.0,Mean Creek,"[{'cast_id': 1, 'character': 'Sam Merric', 'cr...","[{'credit_id': '52fe44d99251416c750430b9', 'de...",603943
2727,2728,500000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",en,"[{'name': 'Rook Films', 'id': 16853}, {'name':...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",9/2/11,95.0,Kill List,"[{'cast_id': 4, 'character': 'Jay', 'credit_id...","[{'credit_id': '52fe48dbc3a368484e10e88d', 'de...",171760
2732,2733,5000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 878, 'na...",en,NaN,NaN,2/1/91,90.0,Meet the Applegates,"[{'cast_id': 4, 'character': 'Richard P. Apple...","[{'credit_id': '52fe493fc3a368484e121a95', 'de...",485772
2758,2759,784170,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"[{'name': 'Gainsborough Pictures', 'id': 1221}...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",4/6/48,80.0,Miranda,"[{'cast_id': 5, 'character': 'Miranda Trewella...","[{'credit_id': '52fe4b08c3a36847f81eeab5', 'de...",834332
2768,2769,225000,"[{'id': 10769, 'name': 'Foreign'}, {'id': 99, ...",en,"[{'name': 'Apostle Productions', 'id': 23648}]","[{'iso_3166_1': 'US', 'name': 'United States o...",11/1/12,86.0,Burn,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe4c71c3a368484e1b893f', 'de...",111300
2786,2787,17000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,"[{'name': 'Coatwolf Productions', 'id': 7313}]","[{'iso_3166_1': 'US', 'name': 'United States o...",4/16/11,106.0,Bellflower,"[{'cast_id': 1002, 'character': 'Woodrow', 'cr...","[{'credit_id': '52fe4970c3a36847f819a227', 'de...",101236
2811,2812,11500000,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",en,"[{'name': 'Battleplan Productions', 'id': 2108...","[{'iso_3166_1': 'US', 'name': 'United States o...",12/19/08,108.0,Nothing But the Truth,"[{'cast_id': 1, 'character': 'Rachel Armstrong...","[{'credit_id': '52fe460b9251416c7506b1df', 'de...",186702
2812,2813,2200000,"[{'id': 18, 'name': 'Drama'}]",hu,"[{'name': 'Proton Cinema', 'id': 17173}, {'nam...","[{'iso_3166_1': 'HU', 'name': 'Hungary'}, {'is...",6/12/14,121.0,White God,"[{'cast_id': 1, 'character': 'Lili', 'credit_i...","[{'credit_id': '5378b097c3a3685525003bfe', 'de...",282358


- We now do the same for the movies with revenue under 1M$

In [ ]:
df.loc[df['title'] == "Henry's Crime", 'revenue'] = 2169799
df.loc[df['title'] == "31", 'revenue'] = 850419
df.loc[df['title'] == "Ten Canoes", 'revenue'] = 3360455
df.loc[df['title'] == "The Wild Bunch", 'revenue'] = 10500000
df.loc[df['title'] == "Happy Family", 'revenue'] = 1271669
df.loc[df['title'] == "Aquarius", 'revenue'] = 3085977
df.loc[df['title'] == "Adore", 'revenue'] = 1575749
df.loc[df['title'] == "Enter the Void", 'revenue'] = 1467278
df.loc[df['title'] == "Shadow Dancer", 'revenue'] = 2255291
df.loc[df['title'] == "Life of Crime", 'revenue'] = 1456675
df.loc[df['title'] == "Beyond the Mat", 'budget'] = 500000
df.loc[df['title'] == "Beyond the Mat", 'revenue'] = 2053648 # Beyond the Mat's revenue and budget were switched
df.loc[df['title'] == "The Monk", 'revenue'] = 1809527
df.loc[df['title'] == "The Face of Love", 'revenue'] = 1806548
df.loc[df['title'] == "Man-Thing", 'revenue'] = 1123136
df.loc[df['title'] == "I Spit on Your Grave", 'revenue'] = 1278650
df.loc[df['title'] == "My Life as a Zucchini", 'revenue'] = 5873157
df.loc[df['title'] == "Burn", 'revenue'] = 1109276

- We erase The Informers because special conditions involved this movie, as for a 18M$ Budget, it was only released for 3 days, not reaching a expected box office. Dark Tide was also a major flop.

In [ ]:
df = df.loc[~((df['title'] == 'The Informers'))]

In [ ]:
df = df.loc[~((df['revenue'] < 1000000))]

#### Genres

In [ ]:
df = df.dropna(subset=['genres'])

In [ ]:
df['genres'] = df['genres'].apply(lambda x:re.findall("'name': \'(.+?)\'", x))

In [ ]:
df.genres[0:20]

0                                         [Comedy]
1                 [Comedy, Drama, Family, Romance]
2                                          [Drama]
3                                [Thriller, Drama]
5                   [Animation, Adventure, Family]
6                               [Horror, Thriller]
9                                  [Comedy, Music]
10                                         [Drama]
12                                         [Drama]
13                                 [Comedy, Crime]
14    [Action, Thriller, Science Fiction, Mystery]
16                              [Horror, Thriller]
18                               [Comedy, Romance]
19                       [Action, Thriller, Crime]
20            [Adventure, Family, Science Fiction]
21                              [Horror, Thriller]
26               [Crime, Drama, Mystery, Thriller]
27                        [Drama, Comedy, Romance]
28                                     [Animation]
29            [Action, Adventur

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

def binarize_multilabel_columns(df, columns):
    mlb = MultiLabelBinarizer()
    df_new = df.copy()  # Copiar el DataFrame original
    
    for column in columns:
        # Transformar la columna con MultiLabelBinarizer
        column_encoded = mlb.fit_transform(df[column])
        
        # Crear un DataFrame con las columnas binarias
        column_encoded_df = pd.DataFrame(column_encoded, columns=mlb.classes_)
        
        # Concatenar el DataFrame original con el DataFrame de columnas binarias
        df_new = pd.concat([df_new.reset_index(drop=True), column_encoded_df.reset_index(drop=True)], axis=1)
    
    return df_new

In [ ]:
columns_to_binarize = ['genres']

# Aplicar la función para binarizar las columnas especificadas
df = binarize_multilabel_columns(df, columns_to_binarize)

- Now I'm aiming to make a feature for the most successful genres

In [ ]:
from collections import Counter

genres_list = df["genres"].explode().tolist()
genresDict = Counter(genres_list)

In [ ]:
genresDict

Counter({'Drama': 966,
         'Comedy': 674,
         'Thriller': 585,
         'Action': 582,
         'Romance': 373,
         'Adventure': 365,
         'Crime': 345,
         'Science Fiction': 227,
         'Horror': 219,
         'Family': 194,
         'Fantasy': 181,
         'Mystery': 167,
         'Animation': 103,
         'History': 96,
         'War': 76,
         'Music': 64,
         'Western': 29,
         'Documentary': 14,
         'Foreign': 6,
         'TV Movie': 1})

In [ ]:
from collections import defaultdict, OrderedDict

# Crear un defaultdict para almacenar los ingresos totales y el conteo de películas por género
topGenreDict = defaultdict(lambda: [0, 0])

# Iterar sobre las filas del DataFrame
for _, row in df.iterrows():
    revenue = row["revenue"]
    genres = row["genres"]
    
    for genre in genres:
        topGenreDict[genre][0] += revenue
        topGenreDict[genre][1] += 1

# Calcular el promedio de ingresos por género
average_revenue_per_genre = {}
for genre in topGenreDict:
    total_revenue, count = topGenreDict[genre]
    if count > 0:
        average_revenue_per_genre[genre] = total_revenue / count
    else:
        average_revenue_per_genre[genre] = 0  # Evitar división por cero si no hay películas

# Ordenar topGenreDict por los ingresos totales en orden descendente
topGenreDict = OrderedDict(sorted(topGenreDict.items(), key=lambda x: x[1][0], reverse=True))

# Mostrar los géneros en orden de ingresos totales
for genre in topGenreDict:
    print(f"Género: {genre}, Total Ingresos: {topGenreDict[genre][0]}, Cantidad Películas: {topGenreDict[genre][1]}")

# También puedes mostrar el promedio de ingresos por género
for genre, avg_revenue in average_revenue_per_genre.items():
    print(f"Género: {genre}, Ingreso Promedio: {avg_revenue}")

Género: Action, Total Ingresos: 80350752217, Cantidad Películas: 582
Género: Adventure, Total Ingresos: 75318286099, Cantidad Películas: 365
Género: Drama, Total Ingresos: 65290574679, Cantidad Películas: 966
Género: Comedy, Total Ingresos: 57393328715, Cantidad Películas: 674
Género: Thriller, Total Ingresos: 52457620719, Cantidad Películas: 585
Género: Science Fiction, Total Ingresos: 34977222585, Cantidad Películas: 227
Género: Family, Total Ingresos: 34434324592, Cantidad Películas: 194
Género: Fantasy, Total Ingresos: 33360985293, Cantidad Películas: 181
Género: Romance, Total Ingresos: 28207087693, Cantidad Películas: 373
Género: Crime, Total Ingresos: 26538088536, Cantidad Películas: 345
Género: Animation, Total Ingresos: 20044566910, Cantidad Películas: 103
Género: Horror, Total Ingresos: 15564605478, Cantidad Películas: 219
Género: Mystery, Total Ingresos: 12716156739, Cantidad Películas: 167
Género: War, Total Ingresos: 4951619965, Cantidad Películas: 76
Género: History, Tota

In [ ]:
topGenreDict

OrderedDict([('Action', [80350752217, 582]),
             ('Adventure', [75318286099, 365]),
             ('Drama', [65290574679, 966]),
             ('Comedy', [57393328715, 674]),
             ('Thriller', [52457620719, 585]),
             ('Science Fiction', [34977222585, 227]),
             ('Family', [34434324592, 194]),
             ('Fantasy', [33360985293, 181]),
             ('Romance', [28207087693, 373]),
             ('Crime', [26538088536, 345]),
             ('Animation', [20044566910, 103]),
             ('Horror', [15564605478, 219]),
             ('Mystery', [12716156739, 167]),
             ('War', [4951619965, 76]),
             ('History', [4870363805, 96]),
             ('Music', [4021949766, 64]),
             ('Western', [2047839318, 29]),
             ('Documentary', [197220094, 14]),
             ('TV Movie', [42000000, 1]),
             ('Foreign', [17288952, 6])])

In [ ]:
genre_index_dict = {genre: index for index, genre in enumerate(topGenreDict.keys())}

def getGenreRank(genres):
    total_rank = sum(genre_index_dict[g] for g in genres)
    return total_rank / len(genres)

- All the dictionaries are saved in the folder imports for future purposes in the project presentation.

In [ ]:
with open('topGenreDict.pkl', 'wb') as f:
    pickle.dump(topGenreDict, f)
with open('genre_index_dict.pkl', 'wb') as f:
    pickle.dump(genre_index_dict, f)

In [ ]:
df['genre_rank'] = df['genres'].apply(lambda x: getGenreRank(x) if len(x) > 0 else 0)

In [ ]:
df

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,genre_rank
0,1,14000000,[Comedy],en,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,Hot Tub Time Machine 2,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.000000
1,2,40000000,"[Comedy, Drama, Family, Romance]",en,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,The Princess Diaries 2: Royal Engagement,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,4.750000
2,3,3300000,[Drama],en,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,Whiplash,"[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.000000
3,4,1200000,"[Thriller, Drama]",hi,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,Kahaani,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,3.000000
4,6,8000000,"[Animation, Adventure, Family]",en,NaN,NaN,8/6/87,83.0,Pinocchio and the Emperor of the Night,"[{'cast_id': 6, 'character': 'Pinocchio (voice...","[{'credit_id': '52fe46f49251416c9106558b', 'de...",3261638,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,2993,1135654,"[Action, Adventure, Drama, Family, Fantasy]",en,"[{'name': 'Douglas Fairbanks Pictures', 'id': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",3/18/24,149.0,The Thief of Bagdad,"[{'cast_id': 3, 'character': 'The Thief of Bag...","[{'credit_id': '52fe45bec3a368484e06c70b', 'de...",1213880,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,3.200000
1991,2994,60000000,"[Comedy, Drama]",en,"[{'name': 'DreamWorks SKG', 'id': 27}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",6/17/04,128.0,The Terminal,"[{'cast_id': 4, 'character': 'Viktor Navorski'...","[{'credit_id': '52fe4259c3a36847f8017699', 'de...",219417255,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.500000
1992,2998,65000000,"[Crime, Action, Mystery, Thriller]",en,"[{'name': 'New Line Cinema', 'id': 12}, {'name...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/11/96,120.0,The Long Kiss Goodnight,"[{'cast_id': 10, 'character': 'Samantha Caine ...","[{'credit_id': '52fe443a9251416c7502d579', 'de...",89456761,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,6.250000
1993,2999,42000000,"[Comedy, Romance]",en,"[{'name': 'Jersey Films', 'id': 216}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",1/16/04,90.0,Along Came Polly,"[{'cast_id': 8, 'character': 'Reuben Feffer', ...","[{'credit_id': '556f817b9251410866000a63', 'de...",171963386,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5.500000


#### Cast

In [ ]:
df = df.dropna(subset=['cast'])

In [ ]:
df[df.cast == '[]'].head()

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,crew,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,genre_rank


- We drop the rows with cast being '[]'

In [ ]:
df = df[df['cast'] != '[]']

In [ ]:
import ast 

def extraer(dict_string):
    list_of_dicts = ast.literal_eval(dict_string)
    names = [dic['name'] for dic in list_of_dicts]
    return names

In [ ]:
df['cast'] = df['cast'].apply(extraer)

In [ ]:
df.cast[0:25]

0     [Rob Corddry, Craig Robinson, Clark Duke, Adam...
1     [Anne Hathaway, Julie Andrews, H√©ctor Elizond...
2     [Miles Teller, J.K. Simmons, Melissa Benoist, ...
3     [Vidya Balan, Nawazuddin Siddiqui, Parambrata ...
4     [Scott Grimes, Tom Bosley, Rickie Lee Jones, J...
5     [Jeffrey Dean Morgan, Natasha Calis, Madison D...
6     [Bob Balaban, Christopher Guest, John Michael ...
7     [Sylvester Stallone, Talia Shire, Burt Young, ...
8     [Kevin Spacey, Annette Bening, Thora Birch, We...
9     [John Travolta, Uma Thurman, Vince Vaughn, Ced...
10    [Tom Cruise, Colin Farrell, Samantha Morton, M...
11    [Rachel McAdams, Cillian Murphy, Brian Cox, Ja...
12    [Felicity Jones, Ed Westwick, Brooke Shields, ...
13    [Jason Statham, Amber Valletta, Kate Nauta, Al...
14    [Gary Oldman, William Hurt, Matt LeBlanc, Mimi...
15    [Dana Kimmell, Paul Kratka, Tracie Savage, Jef...
16    [Steve Martin, Campbell Scott, Ben Gazzara, Re...
17    [Daniel Radcliffe, Zoe Kazan, Rafe Spall, 

In [ ]:
def extraer_actores(lista_de_actores):
    actor_principal = lista_de_actores[0] if len(lista_de_actores) > 0 else None
    actor_reparto_1 = lista_de_actores[1] if len(lista_de_actores) > 1 else None
    actor_reparto_2 = lista_de_actores[2] if len(lista_de_actores) > 2 else None
    return actor_principal, actor_reparto_1, actor_reparto_2

df[['lead_actor', 'supporting_actor_1', 'supporting_actor_2']] = df['cast'].apply(
    lambda x: pd.Series(extraer_actores(x))
)

In [ ]:
df = df.dropna(subset=['supporting_actor_1', 'supporting_actor_2'])

- Let's see the most successful lead actors:

In [ ]:
# Diccionario por defecto para almacenar ingresos totales y cantidad de películas por actor
topActorDict = defaultdict(lambda: [0, 0])

# Iterar sobre las filas del DataFrame
for _, row in df.iterrows():
    revenue = row["revenue"]
    actors = row["cast"]  # Suponiendo que "cast" es una lista de nombres de actores
    
    for actor in actors:
        topActorDict[actor][0] += revenue
        topActorDict[actor][1] += 1

# Filtrar actores que han participado en más de 5 películas
filtered_actors = {actor: stats for actor, stats in topActorDict.items() if stats[1] > 5}

# Calcular la recaudación promedio por actor
average_revenue_per_actor = {}
for actor, (total_revenue, count) in filtered_actors.items():
    average_revenue_per_actor[actor] = total_revenue / count

# Ordenar actores por ingreso promedio en orden descendente
sorted_actors = sorted(average_revenue_per_actor.items(), key=lambda x: x[1], reverse=True)

# Construir topActorDict solo con nombre del actor e ingreso promedio
topActorDict = OrderedDict(sorted_actors)

# Mostrar la recaudación promedio por actor
for actor, avg_revenue in topActorDict.items():
    print(f"Actor: {actor}, Ingreso Promedio: {avg_revenue}")

Actor: Stan Lee, Ingreso Promedio: 856190649.875
Actor: Luke Evans, Ingreso Promedio: 684166188.8333334
Actor: Jess Harnell, Ingreso Promedio: 653163711.25
Actor: Orlando Bloom, Ingreso Promedio: 650910908.7142857
Actor: Andy Serkis, Ingreso Promedio: 627736137.8888888
Actor: Ian McKellen, Ingreso Promedio: 621432050.5454545
Actor: Jed Brophy, Ingreso Promedio: 612949691.3333334
Actor: John DiMaggio, Ingreso Promedio: 582649642.8333334
Actor: Hugo Weaving, Ingreso Promedio: 582574341.5454545
Actor: Emma Watson, Ingreso Promedio: 581954504.8333334
Actor: Christopher Lee, Ingreso Promedio: 576116584.1
Actor: Tyrese Gibson, Ingreso Promedio: 566860816.0
Actor: Peter Cullen, Ingreso Promedio: 561860039.8333334
Actor: Jeremy Renner, Ingreso Promedio: 516149281.3333333
Actor: Shia LaBeouf, Ingreso Promedio: 511103429.125
Actor: Josh Duhamel, Ingreso Promedio: 499268644.14285713
Actor: Alan Tudyk, Ingreso Promedio: 484085879.2222222
Actor: Alan Rickman, Ingreso Promedio: 483482554.28571427
Ac

In [ ]:
topActorDict

OrderedDict([('Stan Lee', 856190649.875),
             ('Luke Evans', 684166188.8333334),
             ('Jess Harnell', 653163711.25),
             ('Orlando Bloom', 650910908.7142857),
             ('Andy Serkis', 627736137.8888888),
             ('Ian McKellen', 621432050.5454545),
             ('Jed Brophy', 612949691.3333334),
             ('John DiMaggio', 582649642.8333334),
             ('Hugo Weaving', 582574341.5454545),
             ('Emma Watson', 581954504.8333334),
             ('Christopher Lee', 576116584.1),
             ('Tyrese Gibson', 566860816.0),
             ('Peter Cullen', 561860039.8333334),
             ('Jeremy Renner', 516149281.3333333),
             ('Shia LaBeouf', 511103429.125),
             ('Josh Duhamel', 499268644.14285713),
             ('Alan Tudyk', 484085879.2222222),
             ('Alan Rickman', 483482554.28571427),
             ('Lee Pace', 482942322.6666667),
             ('Paul Bettany', 476045058.125),
             ('Chris Evans', 4653364

In [ ]:
actor_rank_dict = {} 

# Asignar el ranking a cada actor en actor_rank_dict
for idx, actor in enumerate(sorted(topActorDict, key=topActorDict.get, reverse=True)):
    actor_rank_dict[actor] = idx + 1

# Función para calcular el ranking promedio de los actores en el elenco de una película
def calculate_actor_ranking(actors):
    if isinstance(actors, list):
        rankings = []
        for actor in actors:
            if actor in actor_rank_dict:
                rankings.append(actor_rank_dict[actor])
            else:
                rankings.append(len(topActorDict) + 1)  # Usar el peor ranking si el actor no está en el diccionario
        return sum(rankings) / len(rankings) if len(rankings) > 0 else len(topActorDict) + 1
    else:
        return len(topActorDict) + 1

In [ ]:
df['actors'] = df['cast'].apply(lambda x: x[:5])
df['actor_rank'] = df['actors'].apply(calculate_actor_ranking)

In [ ]:
actor_rank_dict

{'Stan Lee': 1,
 'Luke Evans': 2,
 'Jess Harnell': 3,
 'Orlando Bloom': 4,
 'Andy Serkis': 5,
 'Ian McKellen': 6,
 'Jed Brophy': 7,
 'John DiMaggio': 8,
 'Hugo Weaving': 9,
 'Emma Watson': 10,
 'Christopher Lee': 11,
 'Tyrese Gibson': 12,
 'Peter Cullen': 13,
 'Jeremy Renner': 14,
 'Shia LaBeouf': 15,
 'Josh Duhamel': 16,
 'Alan Tudyk': 17,
 'Alan Rickman': 18,
 'Lee Pace': 19,
 'Paul Bettany': 20,
 'Chris Evans': 21,
 'Thomas Kretschmann': 22,
 'Stellan Skarsg√•rd': 23,
 'Bonnie Hunt': 24,
 'John Rhys-Davies': 25,
 'Judi Dench': 26,
 'Henry Goodman': 27,
 'Christopher Fairbank': 28,
 'Scarlett Johansson': 29,
 'Marton Csokas': 30,
 'Denis Leary': 31,
 'Sean Bean': 32,
 'Glenn Morshower': 33,
 'Kevin McNally': 34,
 'Jenny Agutter': 35,
 'Elijah Wood': 36,
 'Idris Elba': 37,
 'Tom Kenny': 38,
 'Vin Diesel': 39,
 'Ray Winstone': 40,
 'Helena Bonham Carter': 41,
 'Naomie Harris': 42,
 'Cameron Bright': 43,
 'Maggie Smith': 44,
 'Cate Blanchett': 45,
 'Robbie Coltrane': 46,
 'John Ratzenbe

In [ ]:
with open('topActorDict.pkl', 'wb') as f:
    pickle.dump(topActorDict, f)
with open('actor_rank_dict.pkl', 'wb') as f:
    pickle.dump(actor_rank_dict, f)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1978 entries, 0 to 1994
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1978 non-null   int64  
 1   budget                1978 non-null   int64  
 2   genres                1978 non-null   object 
 3   original_language     1978 non-null   object 
 4   production_companies  1941 non-null   object 
 5   production_countries  1971 non-null   object 
 6   release_date          1978 non-null   object 
 7   runtime               1978 non-null   float64
 8   title                 1978 non-null   object 
 9   cast                  1978 non-null   object 
 10  crew                  1978 non-null   object 
 11  revenue               1978 non-null   int64  
 12  Action                1978 non-null   int64  
 13  Adventure             1978 non-null   int64  
 14  Animation             1978 non-null   int64  
 15  Comedy                1978

In [ ]:
# Función para calcular la recaudación media de cada actor que ha participado en al menos 3 películas
def calculate_average_revenue_per_actor(df):
    actor_revenue = Counter()
    actor_count = Counter()
    
    for index, row in df.iterrows():
        cast = row['cast']
        revenue = row['revenue']
        for actor in cast:
            actor_revenue[actor] += revenue
            actor_count[actor] += 1
    
    # Filtrar actores que han participado en menos de 3 películas
    actors_to_remove = [actor for actor in actor_count if actor_count[actor] < 3]
    for actor in actors_to_remove:
        del actor_revenue[actor]
    
    average_revenue = {actor: actor_revenue[actor] / actor_count[actor] for actor in actor_revenue}
    
    return average_revenue

# Calcular la recaudación media de cada actor que ha participado en al menos 3 películas
average_revenue_per_actor = calculate_average_revenue_per_actor(df)

# Convertir a DataFrame y ordenar por recaudación media en orden descendente
average_revenue_df = pd.DataFrame(list(average_revenue_per_actor.items()), columns=['actor', 'average_revenue'])
top_100_actors = average_revenue_df.nlargest(100, 'average_revenue')

# Crear un conjunto de los top 100 actores por recaudación media
top_100_actors_set = set(top_100_actors['actor'])

# Función para contar cuántos actores del top 100 están en cada película
def count_top_100_actors(cast):
    return sum(1 for actor in cast if actor in top_100_actors_set)

# Aplicar la función y crear la nueva columna 'top_100_actor_count'
df['top_100_actor_count'] = df['actors'].apply(lambda x: count_top_100_actors(x))

In [ ]:
with open('top_100_actors_set.pkl', 'wb') as f:
    pickle.dump(top_100_actors_set, f)

In [ ]:
top_100_actors

,actor,average_revenue
3221,Keith Richards,9.892379e+08
1577,Cobie Smulders,9.343306e+08
2172,Karl Farrer,9.161677e+08
3212,Greg Ellis,8.872417e+08
2603,John Brotherton,8.658594e+08
...,...,...
2091,Alan Rickman,4.834826e+08
2584,Geraldine Somerville,4.833649e+08
1657,Tom Everett,4.830250e+08
1124,Lee Pace,4.829423e+08


Now we have a feature that bears in mind the most successful actors, and for example, the one who leads this feature with 4 actors in "The Avengers"

#### Crew

- Falta separar esto por directores o guionistas

In [ ]:
df = df.dropna(subset=['crew'])

In [ ]:
import ast

def extract_director(credits):
    data = ast.literal_eval(credits)
    director = next((dic['name'] for dic in data if dic['job'] == 'Director'), None)
    return director

def extract_writer(credits):
    data = ast.literal_eval(credits)
    writer = next((dic['name'] for dic in data if dic['department'] == 'Writing'), None)
    return writer

In [ ]:
df['director'] = df['crew'].apply(extract_director)
df['writer'] = df['crew'].apply(extract_writer)

In [ ]:
df = df.dropna(subset=['writer'])

- We modify some wrong written names:

In [ ]:
df['director'] = df['director'].replace('Alejandro Gonz√°lez I√±√°rritu', 'Alejandro González Iñárritu')

df['writer'] = df['writer'].replace('Alejandro Gonz√°lez I√±√°rritu', 'Alejandro González Iñárritu')

In [ ]:
df['director'] = df['director'].astype(str)
df['writer'] = df['writer'].astype(str)

In [ ]:
# Diccionario por defecto para almacenar ingresos totales y cantidad de películas por director
directorRevenueDict = defaultdict(lambda: [0, 0])

# Iterar sobre las filas del DataFrame
for _, row in df.iterrows():
    revenue = row["revenue"]
    director = row["director"]
    
    directorRevenueDict[director][0] += revenue
    directorRevenueDict[director][1] += 1

# Calcular la recaudación promedio por director
average_revenue_per_director = {}
for director, (total_revenue, count) in directorRevenueDict.items():
    if count > 0:
        average_revenue_per_director[director] = total_revenue / count
    else:
        average_revenue_per_director[director] = 0  # Evitar división por cero si no hay películas

# Ordenar por ingreso promedio (orden descendente)
sorted_directors = sorted(average_revenue_per_director.items(), key=lambda x: x[1], reverse=True)

# Construir topDirectorDict solo con nombre del director e ingreso promedio
topDirectorDict = OrderedDict((director, avg_revenue) for director, avg_revenue in sorted_directors)

# Mostrar la recaudación promedio por director
for director, avg_revenue in topDirectorDict.items():
    print(f"Director: {director}, Ingreso Promedio: {avg_revenue}")

Director: Joss Whedon, Ingreso Promedio: 1462480802.0
Director: Byron Howard, Ingreso Promedio: 1023784195.0
Director: David Yates, Ingreso Promedio: 933959197.0
Director: James Gunn, Ingreso Promedio: 818372385.0
Director: Roger Allers, Ingreso Promedio: 788241776.0
Director: Tim Miller, Ingreso Promedio: 783112979.0
Director: Robert Stromberg, Ingreso Promedio: 758539785.0
Director: George Lucas, Ingreso Promedio: 749699164.0
Director: Alfonso Cuar√≥n, Ingreso Promedio: 716392705.0
Director: Anthony Russo, Ingreso Promedio: 714766572.0
Director: Carlos Saldanha, Ingreso Promedio: 682605344.0
Director: Zack Snyder, Ingreso Promedio: 662845518.0
Director: Peter Jackson, Ingreso Promedio: 635720590.5714285
Director: Andrew Stanton, Ingreso Promedio: 611340616.3333334
Director: Dean DeBlois, Ingreso Promedio: 609123048.0
Director: Rob Marshall, Ingreso Promedio: 603978382.0
Director: Andrew Adamson, Ingreso Promedio: 584229185.0
Director: David Silverman, Ingreso Promedio: 527068851.0
Di

In [ ]:
topDirectorDict

OrderedDict([('Joss Whedon', 1462480802.0),
             ('Byron Howard', 1023784195.0),
             ('David Yates', 933959197.0),
             ('James Gunn', 818372385.0),
             ('Roger Allers', 788241776.0),
             ('Tim Miller', 783112979.0),
             ('Robert Stromberg', 758539785.0),
             ('George Lucas', 749699164.0),
             ('Alfonso Cuar√≥n', 716392705.0),
             ('Anthony Russo', 714766572.0),
             ('Carlos Saldanha', 682605344.0),
             ('Zack Snyder', 662845518.0),
             ('Peter Jackson', 635720590.5714285),
             ('Andrew Stanton', 611340616.3333334),
             ('Dean DeBlois', 609123048.0),
             ('Rob Marshall', 603978382.0),
             ('Andrew Adamson', 584229185.0),
             ('David Silverman', 527068851.0),
             ('Brad Bird', 511769931.3333333),
             ('Tom McGrath', 498814908.0),
             ('Christopher Nolan', 496151142.6666667),
             ('Francis Lawrence', 489

In [ ]:
# Función para calcular la recaudación media de cada director que ha dirigido al menos 3 películas
def calculate_average_revenue_per_director(df):
    director_revenue = {}
    director_count = {}
    
    for index, row in df.iterrows():
        director = row['director']
        revenue = row['revenue']
        if director in director_revenue:
            director_revenue[director] += revenue
            director_count[director] += 1
        else:
            director_revenue[director] = revenue
            director_count[director] = 1
    
    # Filtrar directores que han dirigido menos de 3 películas
    for director in list(director_revenue.keys()):
        if director_count[director] < 3:
            del director_revenue[director]
            del director_count[director]
    
    average_revenue = {director: director_revenue[director] / director_count[director] for director in director_revenue}
    
    return average_revenue

# Calcular la recaudación media de cada director que ha dirigido al menos 3 películas
average_revenue_per_director = calculate_average_revenue_per_director(df)

# Convertir a DataFrame y ordenar por recaudación media en orden descendente
average_revenue_dir_df = pd.DataFrame(list(average_revenue_per_director.items()), columns=['director', 'average_revenue'])
top_25_directors = average_revenue_dir_df.nlargest(25, 'average_revenue')

# Crear un conjunto de los top 25 directores por recaudación media
top_25_directors_set = set(top_25_directors['director'])

# Función para determinar si la película tiene alguno de los top 25 directores
def has_top_25_director(director):
    return 1 if director in top_25_directors_set else 0

# Aplicar la función y crear la nueva columna 'has_top_25_director'
df['has_top_25_director'] = df['director'].apply(lambda x: has_top_25_director(x))

In [ ]:
with open('topDirectorDict.pkl', 'wb') as f:
    pickle.dump(topDirectorDict, f)
with open('top_25_directors_set.pkl', 'wb') as f:
    pickle.dump(top_25_directors_set, f)

In [ ]:
top_25_directors

,director,average_revenue
12,Carlos Saldanha,6.826053e+08
28,Peter Jackson,6.357206e+08
41,Andrew Stanton,6.113406e+08
164,Brad Bird,5.117699e+08
81,Christopher Nolan,4.961511e+08
58,Francis Lawrence,4.898873e+08
99,Michael Bay,4.883441e+08
112,Bill Condon,4.754597e+08
83,Gore Verbinski,4.715684e+08
156,John Lasseter,4.651299e+08


In [ ]:
director_rank_dict = {} 

# Asignar el ranking a cada director en director_rank_dict
for idx, director in enumerate(sorted(topDirectorDict, key=topDirectorDict.get, reverse=True)):
    director_rank_dict[director] = idx + 1

# Función para calcular el ranking promedio del director de una película
def calculate_director_ranking(director):
    if isinstance(director, str):
        # Devuelve el ranking del director si existe, de lo contrario devuelve el peor ranking posible
        return director_rank_dict.get(director, len(topDirectorDict) + 1)
    else:
        return len(topDirectorDict) + 1

# Aplicar la función y crear la nueva columna 'director_ranking'
df['director_rank'] = df['director'].apply(calculate_director_ranking)

In [ ]:
with open('director_rank_dict.pkl', 'wb') as f:
    pickle.dump(director_rank_dict, f)

In [ ]:
# Diccionario por defecto para almacenar ingresos totales y cantidad de películas por escritor
writerRevenueDict = defaultdict(lambda: [0, 0])

# Iterar sobre las filas del DataFrame
for _, row in df.iterrows():
    revenue = row["revenue"]
    writer = row["writer"]
    
    writerRevenueDict[writer][0] += revenue
    writerRevenueDict[writer][1] += 1

# Calcular la recaudación promedio por escritor
average_revenue_per_writer = {}
for writer, (total_revenue, count) in writerRevenueDict.items():
    if count > 0:
        average_revenue_per_writer[writer] = total_revenue / count
    else:
        average_revenue_per_writer[writer] = 0  # Evitar división por cero si no hay películas

# Ordenar por ingreso promedio (orden descendente)
sorted_writers = sorted(average_revenue_per_writer.items(), key=lambda x: x[1], reverse=True)

# Construir topWriterDict solo con nombre del escritor e ingreso promedio
topWriterDict = OrderedDict((writer, avg_revenue) for writer, avg_revenue in sorted_writers)

# Mostrar la recaudación promedio por escritor
for writer, avg_revenue in topWriterDict.items():
    print(f"Escritor: {writer}, Ingreso Promedio: {avg_revenue}")

Escritor: Stan Lee, Ingreso Promedio: 1085180441.0
Escritor: Jared Bush, Ingreso Promedio: 1023784195.0
Escritor: Fran Walsh, Ingreso Promedio: 1021103568.0
Escritor: J.K. Rowling, Ingreso Promedio: 905323839.5
Escritor: Linda Woolverton, Ingreso Promedio: 892015447.5
Escritor: Chris Morgan, Ingreso Promedio: 832358826.0
Escritor: James Gunn, Ingreso Promedio: 818372385.0
Escritor: Jonathan Roberts, Ingreso Promedio: 788241776.0
Escritor: Ted Elliott, Ingreso Promedio: 779772048.5
Escritor: Alfonso Cuar√≥n, Ingreso Promedio: 716392705.0
Escritor: Melissa Rosenberg, Ingreso Promedio: 704159404.5
Escritor: George Lucas, Ingreso Promedio: 703602613.0
Escritor: Jon Vitti, Ingreso Promedio: 660940780.0
Escritor: Danny Strong, Ingreso Promedio: 653428261.0
Escritor: Jonathan Hales, Ingreso Promedio: 649398328.0
Escritor: Stephen Chbosky, Ingreso Promedio: 647278478.5
Escritor: Michael Berg, Ingreso Promedio: 634971976.5
Escritor: Drew Goddard, Ingreso Promedio: 630161890.0
Escritor: J.R.R. T

In [ ]:
topWriterDict

OrderedDict([('Stan Lee', 1085180441.0),
             ('Jared Bush', 1023784195.0),
             ('Fran Walsh', 1021103568.0),
             ('J.K. Rowling', 905323839.5),
             ('Linda Woolverton', 892015447.5),
             ('Chris Morgan', 832358826.0),
             ('James Gunn', 818372385.0),
             ('Jonathan Roberts', 788241776.0),
             ('Ted Elliott', 779772048.5),
             ('Alfonso Cuar√≥n', 716392705.0),
             ('Melissa Rosenberg', 704159404.5),
             ('George Lucas', 703602613.0),
             ('Jon Vitti', 660940780.0),
             ('Danny Strong', 653428261.0),
             ('Jonathan Hales', 649398328.0),
             ('Stephen Chbosky', 647278478.5),
             ('Michael Berg', 634971976.5),
             ('Drew Goddard', 630161890.0),
             ('J.R.R. Tolkien', 620079928.0),
             ('Mel Gibson', 611899420.0),
             ('Andrew Stanton', 611340616.3333334),
             ('Cressida Cowell', 609123048.0),
           

In [ ]:
# Función para calcular la recaudación media de cada escritor que ha escrito al menos 3 películas
def calculate_average_revenue_per_writer(df):
    writer_revenue = {}
    writer_count = {}
    
    for index, row in df.iterrows():
        writer = row['writer']
        revenue = row['revenue']
        if writer in writer_revenue:
            writer_revenue[writer] += revenue
            writer_count[writer] += 1
        else:
            writer_revenue[writer] = revenue
            writer_count[writer] = 1
    
    # Filtrar escritores que han escrito menos de 3 películas
    for writer in list(writer_revenue.keys()):
        if writer_count[writer] < 3:
            del writer_revenue[writer]
            del writer_count[writer]
    
    average_revenue = {writer: writer_revenue[writer] / writer_count[writer] for writer in writer_revenue}
    
    return average_revenue

# Calcular la recaudación media de cada escritor que ha escrito al menos 3 películas
average_revenue_per_writer = calculate_average_revenue_per_writer(df)

# Convertir a DataFrame y ordenar por recaudación media en orden descendente
average_revenue_wri_df = pd.DataFrame(list(average_revenue_per_writer.items()), columns=['writer', 'average_revenue'])
top_25_writers = average_revenue_wri_df.nlargest(25, 'average_revenue')

# Crear un conjunto de los top 25 escritores por recaudación media
top_25_writers_set = set(top_25_writers['writer'])

# Función para determinar si la película tiene alguno de los top 25 escritores
def has_top_25_writer(writer):
    return 1 if writer in top_25_writers_set else 0

# Aplicar la función y crear la nueva columna 'has_top_15_writer'
df['has_top_25_writer'] = df['writer'].apply(lambda x: has_top_25_writer(x))

In [ ]:
with open('topWriterDict.pkl', 'wb') as f:
    pickle.dump(topWriterDict, f)
with open('top_25_writers_set.pkl', 'wb') as f:
    pickle.dump(top_25_writers_set, f)

In [ ]:
top_25_writers

,writer,average_revenue
73,Stan Lee,1.085180e+09
108,Ted Elliott,7.797720e+08
62,George Lucas,7.036026e+08
17,J.R.R. Tolkien,6.200799e+08
24,Andrew Stanton,6.113406e+08
14,Ehren Kruger,5.533676e+08
49,Christopher Nolan,4.961511e+08
85,Akiva Goldsman,4.329437e+08
44,Peter Jackson,3.997931e+08
23,Ian Fleming,2.822290e+08


In [ ]:
writer_rank_dict = {}

# Asignar el ranking a cada escritor en writer_rank_dict
for idx, writer in enumerate(sorted(topWriterDict, key=topWriterDict.get, reverse=True)):
    writer_rank_dict[writer] = idx + 1

# Función para calcular el ranking promedio del escritor de una película
def calculate_writer_ranking(writer):
    if isinstance(writer, str):
        # Devuelve el ranking del escritor si existe, de lo contrario devuelve el peor ranking posible
        return writer_rank_dict.get(writer, len(topWriterDict) + 1)
    else:
        # Si no es un string, asume que no hay información disponible y devuelve el peor ranking
        return len(topWriterDict) + 1

# Aplicar la función y crear la nueva columna 'writer_ranking'
df['writer_rank'] = df['writer'].apply(calculate_writer_ranking)

In [ ]:
with open('writer_rank_dict.pkl', 'wb') as f:
    pickle.dump(writer_rank_dict, f)

In [ ]:
df = df.drop(['crew'], axis=1)

In [ ]:
df

,id,budget,genres,original_language,production_companies,production_countries,release_date,runtime,title,cast,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,genre_rank,lead_actor,supporting_actor_1,supporting_actor_2,actors,actor_rank,top_100_actor_count,director,writer,has_top_25_director,director_rank,has_top_25_writer,writer_rank
0,1,14000000,[Comedy],en,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,Hot Tub Time Machine 2,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",12314651,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.000000,Rob Corddry,Craig Robinson,Clark Duke,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",746.20,0,Steve Pink,Josh Heald,0,587,0,1015
1,2,40000000,"[Comedy, Drama, Family, Romance]",en,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,The Princess Diaries 2: Royal Engagement,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...",95149435,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,4.750000,Anne Hathaway,Julie Andrews,H√©ctor Elizondo,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...",386.20,0,Garry Marshall,Shonda Rhimes,0,158,0,447
2,3,3300000,[Drama],en,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,Whiplash,"[Miles Teller, J.K. Simmons, Melissa Benoist, ...",13092000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.000000,Miles Teller,J.K. Simmons,Melissa Benoist,"[Miles Teller, J.K. Simmons, Melissa Benoist, ...",735.20,0,Damien Chazelle,Damien Chazelle,0,826,0,630
3,4,1200000,"[Thriller, Drama]",hi,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,Kahaani,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",16000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,3.000000,Vidya Balan,Nawazuddin Siddiqui,Parambrata Chatterjee,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",835.00,0,Sujoy Ghosh,Sujoy Ghosh,0,775,0,950
4,6,8000000,"[Animation, Adventure, Family]",en,NaN,NaN,8/6/87,83.0,Pinocchio and the Emperor of the Night,"[Scott Grimes, Tom Bosley, Rickie Lee Jones, J...",3261638,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5.666667,Scott Grimes,Tom Bosley,Rickie Lee Jones,"[Scott Grimes, Tom Bosley, Rickie Lee Jones, J...",650.25,0,Hal Sutherland,Collodi,0,1066,0,1282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,2993,1135654,"[Action, Adventure, Drama, Family, Fantasy]",en,"[{'name': 'Douglas Fairbanks Pictures', 'id': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",3/18/24,149.0,The Thief of Bagdad,"[Douglas Fairbanks, Snitz Edwards, Charles Bel...",1213880,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,3.200000,Douglas Fairbanks,Snitz Edwards,Charles Belcher,"[Douglas Fairbanks, Snitz Edwards, Charles Bel...",835.00,0,Raoul Walsh,Douglas Fairbanks,0,1164,0,1405
1991,2994,60000000,"[Comedy, Drama]",en,"[{'name': 'DreamWorks SKG', 'id': 27}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",6/17/04,128.0,The Terminal,"[Tom Hanks, Catherine Zeta-Jones, Stanley Tucc...",219417255,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.500000,Tom Hanks,Catherine Zeta-Jones,Stanley Tucci,"[Tom Hanks, Catherine Zeta-Jones, Stanley Tucc...",462.60,0,Steven Spielberg,Jeff Nathanson,1,33,1,89
1992,2998,65000000,"[Crime, Action, Mystery, Thriller]",en,"[{'name': 'New Line Cinema', 'id': 12}, {'name...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/11/96,120.0,The Long Kiss Goodnight,"[Geena Davis, Samuel L. Jackson, Yvonne Zima, ...",89456761,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,6.250000,Geena Davis,Samuel L. Jackson,Yvonne Zima,"[Geena Davis, Samuel L. Jackson, Yvonne Zima, ...",678.60,0,Renny Harlin,Shane Black,0,461,0,429
1993,2999,420000

#### Original Language

In [ ]:
df.original_language.value_counts()

original_language
en    1749
hi      36
fr      29
ru      20
ja      17
es      14
ta      12
zh      11
ko      10
de       8
it       8
cn       7
pt       4
pl       2
nl       2
da       2
sv       2
id       1
ro       1
el       1
nb       1
no       1
he       1
te       1
ml       1
Name: count, dtype: int64

- Most of it is in English, so we're creating a new feature

In [ ]:
df['in_english'] = df.original_language.apply(lambda x: 1 if x == 'en' else 0)

In [ ]:
df.drop(columns=['original_language'], inplace=True)

In [ ]:
df.head()

,id,budget,genres,production_companies,production_countries,release_date,runtime,title,cast,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,genre_rank,lead_actor,supporting_actor_1,supporting_actor_2,actors,actor_rank,top_100_actor_count,director,writer,has_top_25_director,director_rank,has_top_25_writer,writer_rank,in_english
0,1,14000000,[Comedy],"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,Hot Tub Time Machine 2,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",12314651,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.000000,Rob Corddry,Craig Robinson,Clark Duke,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",746.20,0,Steve Pink,Josh Heald,0,587,0,1015,1
1,2,40000000,"[Comedy, Drama, Family, Romance]","[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,The Princess Diaries 2: Royal Engagement,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...",95149435,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,4.750000,Anne Hathaway,Julie Andrews,H√©ctor Elizondo,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...",386.20,0,Garry Marshall,Shonda Rhimes,0,158,0,447,1
2,3,3300000,[Drama],"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,Whiplash,"[Miles Teller, J.K. Simmons, Melissa Benoist, ...",13092000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.000000,Miles Teller,J.K. Simmons,Melissa Benoist,"[Miles Teller, J.K. Simmons, Melissa Benoist, ...",735.20,0,Damien Chazelle,Damien Chazelle,0,826,0,630,1
3,4,1200000,"[Thriller, Drama]",NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,Kahaani,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",16000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,3.000000,Vidya Balan,Nawazuddin Siddiqui,Parambrata Chatterjee,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",835.00,0,Sujoy Ghosh,Sujoy Ghosh,0,775,0,950,0
4,6,8000000,"[Animation, Adventure, Family]",NaN,NaN,8/6/87,83.0,Pinocchio and the Emperor of the Night,"[Scott Grimes, Tom Bosley, Rickie Lee Jones, J...",3261638,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5.666667,Scott Grimes,Tom Bosley,Rickie Lee Jones,"[Scott Grimes, Tom Bosley, Rickie Lee Jones, J...",650.25,0,Hal Sutherland,Collodi,0,1066,0,1282,1


#### Production Companies

In [ ]:
df[df.production_companies.isna()].head()

,id,budget,genres,production_companies,production_countries,release_date,runtime,title,cast,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,genre_rank,lead_actor,supporting_actor_1,supporting_actor_2,actors,actor_rank,top_100_actor_count,director,writer,has_top_25_director,director_rank,has_top_25_writer,writer_rank,in_english
3,4,1200000,"[Thriller, Drama]",NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,Kahaani,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",16000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,3.000000,Vidya Balan,Nawazuddin Siddiqui,Parambrata Chatterjee,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",835.00,0,Sujoy Ghosh,Sujoy Ghosh,0,775,0,950,0
4,6,8000000,"[Animation, Adventure, Family]",NaN,NaN,8/6/87,83.0,Pinocchio and the Emperor of the Night,"[Scott Grimes, Tom Bosley, Rickie Lee Jones, J...",3261638,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5.666667,Scott Grimes,Tom Bosley,Rickie Lee Jones,"[Scott Grimes, Tom Bosley, Rickie Lee Jones, J...",650.25,0,Hal Sutherland,Collodi,0,1066,0,1282,1
33,51,560000,"[Romance, Drama, Comedy]",NaN,"[{'iso_3166_1': 'BE', 'name': 'Belgium'}]",1/30/08,102.0,"Moscow, Belgium","[Barbara Sarafian, Jurgen Delnaet, Johan Helde...",1500000,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,4.333333,Barbara Sarafian,Jurgen Delnaet,Johan Heldenbergh,"[Barbara Sarafian, Jurgen Delnaet, Johan Helde...",835.00,0,Christophe Van Rompaey,Jean-Claude Van Rijckeghem,0,1144,0,1384,0
141,220,2600000,[Documentary],NaN,"[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",7/9/04,105.0,Riding Giants,"[Jeff Clark, Darrick Doerner, Laird Hamilton, ...",3166000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17.000000,Jeff Clark,Darrick Doerner,Laird Hamilton,"[Jeff Clark, Darrick Doerner, Laird Hamilton, ...",835.00,0,Stacy Peralta,Stacy Peralta,0,1072,0,1290,1
176,277,2915739,"[Horror, Fantasy, Thriller]",NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",1/16/09,132.0,Arundhati,"[Sonu Sood, Arjan Bajwa, Manorama, Anushka She...",14000000,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,7.333333,Sonu Sood,Arjan Bajwa,Manorama,"[Sonu Sood, Arjan Bajwa, Manorama, Anushka She...",835.00,0,Kodi Rama Krishna,Chintapalli Ramana,0,808,0,990,0


In [ ]:
df['production_companies'] = df['production_companies'].fillna('')

In [ ]:
df['production_companies'] = df['production_companies'].apply(lambda x:re.findall("'name': \'(.+?)\'", x))

df['num_production_companies'] = df['production_companies'].apply(lambda x: len(x))

In [ ]:
topCompanyDict = defaultdict(lambda: [0, 0])

# Contador de películas por compañía
movie_count = defaultdict(int)

# Iterar sobre las filas del DataFrame
for _, row in df.iterrows():
    revenue = row["revenue"]
    companies = row["production_companies"]
    
    # Actualizar ingresos totales y cantidad de películas por compañía
    for company in companies:
        movie_count[company] += 1
        topCompanyDict[company][0] += revenue
        topCompanyDict[company][1] += 1

# Calcular el promedio de ingresos por compañía y filtrar por más de 3 películas hechas
topCompanyDict = {company: total_revenue / count for company, (total_revenue, count) in topCompanyDict.items() if movie_count[company] > 3}

# Ordenar topCompanyDict por los ingresos medios en orden descendente
topCompanyDict = OrderedDict(sorted(topCompanyDict.items(), key=lambda x: x[1], reverse=True))

In [ ]:
topCompanyDict

OrderedDict([('Marvel Studios', 1055294589.2),
             ('Hasbro Studios', 818674036.75),
             ('Revolution Sun Studios', 655469382.875),
             ('Jerry Bruckheimer Films', 561781952.2857143),
             ('WingNut Films', 559200424.0),
             ('Syncopy', 537824736.5),
             ('Pixar Animation Studios', 532351327.875),
             ('DC Entertainment', 510827266.0),
             ('Temple Hill Entertainment', 506685642.25),
             ('Blue Sky Studios', 458239231.0),
             ('Sunswept Entertainment', 452348692.25),
             ('Lucasfilm', 449454802.14285713),
             ('Moving Picture Company (MPC)', 445917392.0),
             ('Di Bonaventura Pictures', 425142483.3333333),
             ('Walt Disney Feature Animation', 418655834.25),
             ('Twentieth Century Fox Animation', 418150106.0),
             ('Roth Films', 399338626.6),
             ('Dentsu', 379458715.14285713),
             ('Marvel Entertainment', 378937150.75),
     

In [ ]:
df['num_production_companies'].value_counts()

num_production_companies
2     497
1     440
3     401
4     222
5     138
6      96
7      53
0      28
8      28
9      22
10      6
11      4
14      2
12      2
16      1
15      1
Name: count, dtype: int64

In [ ]:
# Función para calcular la recaudación media de cada compañía de producción que ha producido al menos 5 películas
def calculate_average_revenue_per_company(df):
    company_revenue = {}
    company_count = {}
    
    for index, row in df.iterrows():
        companies = row['production_companies']
        revenue = row['revenue']
        for company in companies:
            if company in company_revenue:
                company_revenue[company] += revenue
                company_count[company] += 1
            else:
                company_revenue[company] = revenue
                company_count[company] = 1
    
    # Filtrar compañías que han producido menos de 5 películas
    for company in list(company_revenue.keys()):
        if company_count[company] < 5:
            del company_revenue[company]
            del company_count[company]
    
    average_revenue = {company: company_revenue[company] / company_count[company] for company in company_revenue}
    
    return average_revenue

# Calcular la recaudación media de cada compañía de producción que ha producido al menos 5 películas
average_revenue_per_company = calculate_average_revenue_per_company(df)

# Convertir a DataFrame y ordenar por recaudación media en orden descendente
average_revenue_company_df = pd.DataFrame(list(average_revenue_per_company.items()), columns=['production_company', 'average_revenue'])
top_25_companies = average_revenue_company_df.nlargest(25, 'average_revenue')

# Crear un conjunto de las top 25 compañías de producción por recaudación media
top_25_companies_set = set(top_25_companies['production_company'])

# Función para determinar si la película tiene alguna de las top 25 compañías de producción
def has_top_25_company(companies):
    return 1 if any(company in top_25_companies_set for company in companies) else 0

# Aplicar la función y crear la nueva columna 'has_top_25_company'
df['has_top_25_company'] = df['production_companies'].apply(lambda x: has_top_25_company(x))

In [ ]:
top_25_companies

,production_company,average_revenue
162,Marvel Studios,1.055295e+09
163,Revolution Sun Studios,6.554694e+08
159,Jerry Bruckheimer Films,5.617820e+08
47,WingNut Films,5.592004e+08
67,Pixar Animation Studios,5.323513e+08
24,Blue Sky Studios,4.582392e+08
129,Lucasfilm,4.494548e+08
130,Moving Picture Company (MPC),4.459174e+08
106,Di Bonaventura Pictures,4.251425e+08
25,Twentieth Century Fox Animation,4.181501e+08


In [ ]:
topCompanyDict

OrderedDict([('Marvel Studios', 1055294589.2),
             ('Hasbro Studios', 818674036.75),
             ('Revolution Sun Studios', 655469382.875),
             ('Jerry Bruckheimer Films', 561781952.2857143),
             ('WingNut Films', 559200424.0),
             ('Syncopy', 537824736.5),
             ('Pixar Animation Studios', 532351327.875),
             ('DC Entertainment', 510827266.0),
             ('Temple Hill Entertainment', 506685642.25),
             ('Blue Sky Studios', 458239231.0),
             ('Sunswept Entertainment', 452348692.25),
             ('Lucasfilm', 449454802.14285713),
             ('Moving Picture Company (MPC)', 445917392.0),
             ('Di Bonaventura Pictures', 425142483.3333333),
             ('Walt Disney Feature Animation', 418655834.25),
             ('Twentieth Century Fox Animation', 418150106.0),
             ('Roth Films', 399338626.6),
             ('Dentsu', 379458715.14285713),
             ('Marvel Entertainment', 378937150.75),
     

In [ ]:
company_rank_dict = {}  # Usar un diccionario estándar

# Asignar el ranking a cada compañía en company_rank_dict
for idx, company in enumerate(sorted(topCompanyDict, key=topCompanyDict.get, reverse=True)):
    company_rank_dict[company] = idx + 1

def calculate_company_ranking(companies):
    if isinstance(companies, list):
        rankings = []
        for company in companies:
            rank = company_rank_dict.get(company, len(topCompanyDict) + 1)  # Uso de get con valor por defecto
            rankings.append(rank)
        return sum(rankings) / len(rankings) if len(rankings) > 0 else len(topCompanyDict) + 1
    else:
        return len(topCompanyDict) + 1

In [ ]:
df['company_rank'] = df['production_companies'].apply(calculate_company_ranking)

In [ ]:
with open('topCompanyDict.pkl', 'wb') as f:
    pickle.dump(topCompanyDict, f)
with open('top_25_companies_set.pkl', 'wb') as f:
    pickle.dump(top_25_companies_set, f)
with open('company_rank_dict.pkl', 'wb') as f:
    pickle.dump(company_rank_dict, f)

#### Production Countries

In [ ]:
df.production_countries = df.production_countries.fillna("")

In [ ]:
df['production_countries'] = df['production_countries'].apply(lambda x: re.findall("'name': \'(.+?)\'", x))

df['num_production_countries'] = df['production_countries'].apply(lambda x: len(x))

In [ ]:
topCountryDict = defaultdict(lambda: [0, 0])

for _, row in df.iterrows():
    revenue = row["revenue"]
    countries = row["production_countries"]
    
    temp = min(1, len(countries))
    for i in range(temp):
        country = countries[i]
        topCountryDict[country][0] += revenue
        topCountryDict[country][1] += 1

for country in topCountryDict:
    total_revenue, count = topCountryDict[country]
    topCountryDict[country] = total_revenue / count

topCountryDict = dict(topCountryDict)

In [ ]:
topCountryDict

{'United States of America': 97991928.69796558,
 'India': 26622549.824561402,
 'Austria': 31773513.0,
 'France': 59734570.37974683,
 'Canada': 102926018.58208956,
 'Italy': 60601440.2631579,
 'Belgium': 41031835.428571425,
 'United Kingdom': 132280211.46896552,
 'Japan': 111779318.0,
 'Germany': 114502049.15151516,
 'Hong Kong': 63183315.428571425,
 'Czech Republic': 177435535.44444445,
 'Spain': 58197109.4375,
 'New Zealand': 448102839.1818182,
 'Australia': 77053651.28571428,
 'South Korea': 23391249.777777776,
 'Switzerland': 10350619.0,
 'China': 99776059.04347827,
 'Iceland': 203427584.0,
 'Ireland': 21158368.0,
 'Russia': 16193277.105263159,
 'Brazil': 3024848.5,
 'Netherlands': 1271669.0,
 'Hungary': 89034670.0,
 'Mexico': 67449272.55555555,
 'Greece': 13210581.666666666,
 'Finland': 6811703.666666667,
 'Denmark': 17141241.25,
 'Cambodia': 86234523.0,
 'United Arab Emirates': 65565440.0,
 'Namibia': 4192440.0,
 'South Africa': 27187375.0,
 'Argentina': 13837500.5,
 'Israel': 953

In [ ]:
countryCountDict = defaultdict(int)

for _, row in df.iterrows():
    countries = row["production_countries"]
    for country in countries:
        countryCountDict[country] += 1

countryCountDict = dict(countryCountDict)

In [ ]:
countryCountDict

{'United States of America': 1637,
 'India': 59,
 'Canada': 85,
 'Austria': 4,
 'Germany': 120,
 'United Kingdom': 242,
 'France': 124,
 'Ireland': 12,
 'Italy': 37,
 'Belgium': 13,
 'Czech Republic': 12,
 'Japan': 28,
 'Hong Kong': 21,
 'Spain': 27,
 'New Zealand': 14,
 'Australia': 44,
 'South Korea': 11,
 'Switzerland': 5,
 'Morocco': 3,
 'Luxembourg': 6,
 'Netherlands': 5,
 'China': 28,
 'Iceland': 2,
 'Taiwan': 2,
 'Russia': 27,
 'Brazil': 4,
 'Hungary': 5,
 'Serbia': 1,
 'Mexico': 9,
 'South Africa': 6,
 'Greece': 4,
 'United Arab Emirates': 6,
 'Finland': 5,
 'Denmark': 4,
 'Sweden': 7,
 'Cambodia': 1,
 'Malta': 2,
 'Namibia': 1,
 'Serbia and Montenegro': 1,
 'Argentina': 3,
 'Israel': 2,
 'Qatar': 1,
 'Tunisia': 2,
 'Norway': 3,
 'Bulgaria': 2,
 'Poland': 3,
 'Slovenia': 1,
 'Turkey': 1,
 'Chile': 2,
 'Uruguay': 1,
 'Paraguay': 1,
 'Philippines': 1,
 'Ukraine': 1,
 'Costa Rica': 1,
 'Bahamas': 2,
 'Burkina Faso': 1,
 'Ghana': 1,
 'Romania': 3,
 'Singapore': 1,
 'Indonesia': 1,


- The vast majority is produced in the US, so we're creating a new feature for it

In [ ]:
df['produced_in_US'] = df.production_countries.apply(lambda x: 1 if 'United States of America' in x else 0)

In [ ]:
df['produced_in_US'].value_counts()

produced_in_US
1    1637
0     304
Name: count, dtype: int64

In [ ]:
df.drop(columns=['production_countries'], inplace=True)

#### Release date

- From this column we can extract several useful features as year, season, month, etc.

In [ ]:
df['release_date'] = df['release_date'].apply(str)

In [ ]:
estaciones = {
    1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring',
    5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'
}

def extraer_mes(fecha):
    try:
        # Diccionario de meses
        meses = {
            '1': 'January', '2': 'February', '3': 'March', '4': 'April',
            '5': 'May', '6': 'June', '7': 'July', '8': 'August',
            '9': 'September', '10': 'October', '11': 'November', '12': 'December'
        }
        
        # Extraer el mes de la fecha en formato 'MM/DD/YY'
        mes_num = fecha.split('/')[0]
        
        # Obtener el nombre del mes usando el diccionario
        nombre_mes = meses.get(mes_num, None)
        
        return nombre_mes
    except (AttributeError, IndexError):
        return None
    
def extraer_anio(fecha):
    anio = fecha[-2:]
    if int(anio) <= 24:
        anio = '20' + anio
    else:
        anio = '19' + anio
    return anio
    
def obtener_decada(year):
    if 1920 <= year < 1930:
        return "1920s"
    if 1930 <= year < 1940:
        return "1930s"
    if 1940 <= year < 1950:
        return "1940s"
    if 1950 <= year < 1960:
        return "1950s"
    if 1960 <= year < 1970:
        return "1960s"
    elif 1970 <= year < 1980:
        return "1970s"
    elif 1980 <= year < 1990:
        return "1980s"
    elif 1990 <= year < 2000:
        return "1990s"
    elif 2000 <= year < 2010:
        return "2000s"
    elif 2010 <= year < 2020:
        return "2010s"
    else:
        return "2020s"

def extract_season(date):
    try:
        month_num = int(date.split('/')[0])
        
        if month_num in [3, 4, 5]:
            return 'Spring'
        elif month_num in [6, 7, 8]:
            return 'Summer'
        elif month_num in [9, 10, 11]:
            return 'Autumn'
        elif month_num in [12, 1, 2]:
            return 'Winter'
        else:
            return None  # Return None for invalid month
    except (AttributeError, IndexError, ValueError):
        return None

df['release_month'] = df['release_date'].apply(extraer_mes)
df['release_year'] = df['release_date'].apply(extraer_anio)
df['release_year'] = df['release_year'].apply(int)
df['decade'] = df['release_year'].apply(obtener_decada)
df['season'] = df['release_date'].apply(extract_season)

In [ ]:
df.release_month.value_counts()

release_month
September    215
October      201
December     199
June         173
August       165
April        152
July         150
March        148
February     140
May          138
November     135
January      125
Name: count, dtype: int64

- We extract the different seasons, weeks of the year, months, etc., because the revenue will be different depending on it, if people are more eager to go to the cinema, it's Summer, it's Christmas, etc

- There are some favourite weeks to release, which are the beginning of September and Christmas

In [ ]:
df.head()

,id,budget,genres,production_companies,release_date,runtime,title,cast,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,genre_rank,lead_actor,supporting_actor_1,supporting_actor_2,actors,actor_rank,top_100_actor_count,director,writer,has_top_25_director,director_rank,has_top_25_writer,writer_rank,in_english,num_production_companies,has_top_25_company,company_rank,num_production_countries,produced_in_US,release_month,release_year,decade,season,week_number
0,1,14000000,[Comedy],"[Paramount Pictures, United Artists, Metro-Gol...",2/20/15,93.0,Hot Tub Time Machine 2,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",12314651,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.000000,Rob Corddry,Craig Robinson,Clark Duke,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",746.20,0,Steve Pink,Josh Heald,0,587,0,1015,1,3,0,135.000000,1,1,February,2015,2010s,Winter,8
1,2,40000000,"[Comedy, Drama, Family, Romance]",[Walt Disney Pictures],8/6/04,113.0,The Princess Diaries 2: Royal Engagement,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...",95149435,0,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,4.750000,Anne Hathaway,Julie Andrews,H√©ctor Elizondo,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...",386.20,0,Garry Marshall,Shonda Rhimes,0,158,0,447,1,1,1,23.000000,1,1,August,2004,2000s,Summer,32
2,3,3300000,[Drama],"[Bold Films, Blumhouse Productions, Right of W...",10/10/14,105.0,Whiplash,"[Miles Teller, J.K. Simmons, Melissa Benoist, ...",13092000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2.000000,Miles Teller,J.K. Simmons,Melissa Benoist,"[Miles Teller, J.K. Simmons, Melissa Benoist, ...",735.20,0,Damien Chazelle,Damien Chazelle,0,826,0,630,1,3,0,215.666667,1,1,October,2014,2010s,Autumn,41
3,4,1200000,"[Thriller, Drama]",[],3/9/12,122.0,Kahaani,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",16000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,3.000000,Vidya Balan,Nawazuddin Siddiqui,Parambrata Chatterjee,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",835.00,0,Sujoy Ghosh,Sujoy Ghosh,0,775,0,950,0,0,0,261.000000,1,0,March,2012,2010s,Spring,10
4,6,8000000,"[Animation, Adventure, Family]",[],8/6/87,83.0,Pinocchio and the Emperor of the Night,"[Scott Grimes, Tom Bosley, Rickie Lee Jones, J...",3261638,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5.666667,Scott Grimes,Tom Bosley,Rickie Lee Jones,"[Scott Grimes, Tom Bosley, Rickie Lee Jones, J...",650.25,0,Hal Sutherland,Collodi,0,1066,0,1282,1,0,0,261.000000,0,0,August,1987,1980s,Summer,32


#### Runtime

In [ ]:
df[df.runtime.isna()]

,id,budget,genres,production_companies,release_date,runtime,title,cast,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,genre_rank,lead_actor,supporting_actor_1,supporting_actor_2,actors,actor_rank,top_100_actor_count,director,writer,has_top_25_director,director_rank,has_top_25_writer,writer_rank,in_english,num_production_companies,has_top_25_company,company_rank,num_production_countries,produced_in_US,release_month,release_year,decade,season,week_number


- There is a movie with no Runtime, so we apply the median

In [ ]:
df.runtime = df.runtime.fillna(df.runtime.median())

In [ ]:
print('Movies with no runtime: ',len(df[df.runtime == 0]))

Movies with no runtime:  4


- We also apply the median for them:

In [ ]:
median = df.loc[df['runtime'] > 0, 'runtime'].median()

df['runtime'] = df['runtime'].apply(lambda x: median if x == 0 else x)

- We need to inspect the really long movies too (4 hours), to see if there are outliers

In [ ]:
df[df.runtime > 240]

,id,budget,genres,production_companies,release_date,runtime,title,cast,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,genre_rank,lead_actor,supporting_actor_1,supporting_actor_2,actors,actor_rank,top_100_actor_count,director,writer,has_top_25_director,director_rank,has_top_25_writer,writer_rank,in_english,num_production_companies,has_top_25_company,company_rank,num_production_countries,produced_in_US,release_month,release_year,decade,season,week_number
1265,1923,31115000,"[Drama, History, Romance]","[Twentieth Century Fox Film Corporation, MCL F...",6/12/63,248.0,Cleopatra,"[Elizabeth Taylor, Richard Burton, Rex Harriso...",71000000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,8.0,Elizabeth Taylor,Richard Burton,Rex Harrison,"[Elizabeth Taylor, Richard Burton, Rex Harriso...",830.2,0,Joseph L. Mankiewicz,Carlo Mario Franzero,0,372,0,486,1,3,0,208.666667,3,1,June,1963,1960s,Summer,24


- Cleopatra is a well-known movie I don't feel comfortable dropping, but I'm dropping Carlos due to its very long runtime

In [ ]:
df = df.drop(df[df['title'] == 'Carlos'].index)

- I am also dropping the movies with runtime below 60 min, since they are not considered movies

In [ ]:
df[df.runtime < 60]

,id,budget,genres,production_companies,release_date,runtime,title,cast,revenue,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,genre_rank,lead_actor,supporting_actor_1,supporting_actor_2,actors,actor_rank,top_100_actor_count,director,writer,has_top_25_director,director_rank,has_top_25_writer,writer_rank,in_english,num_production_companies,has_top_25_company,company_rank,num_production_countries,produced_in_US,release_month,release_year,decade,season,week_number


In [ ]:
df = df.drop(df[df['runtime'] < 60].index)

In [ ]:
df.drop(columns=['release_date'], inplace=True)

- Along the years, movies have become longer and longer, so we need to build a ratio in order to bear in mind this difference.

In [ ]:
df['runtime_ratio'] = df['runtime'] / df['release_year']

#### Inflation budget

- It is very important to bear in mind the inflation along the decades, in order to do so, I got the aproximate inflation rates and created a new feature

In [ ]:
# Datos de tasas de inflación por década
inflation_data = {
    'decade': ['1930s', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s'],
    'average_inflation_rate': [0.012, 0.016, 0.022, 0.025, 0.074, 0.051, 0.030, 0.026, 0.018]
}
inflation_df = pd.DataFrame(inflation_data)

# Función para obtener la tasa de inflación promedio por año
def get_inflation_rate(year):
    if 1930 <= year < 1940:
        return 0.012
    elif 1940 <= year < 1950:
        return 0.016
    elif 1950 <= year < 1960:
        return 0.022
    elif 1960 <= year < 1970:
        return 0.025
    elif 1970 <= year < 1980:
        return 0.074
    elif 1980 <= year < 1990:
        return 0.051
    elif 1990 <= year < 2000:
        return 0.030
    elif 2000 <= year < 2010:
        return 0.026
    elif 2010 <= year < 2020:
        return 0.018
    else:
        return 0.02  # Valor predeterminado para años más allá de 2020

df['average_inflation_rate'] = df['release_year'].apply(get_inflation_rate)

In [ ]:
def adjust_for_inflation(budget, release_year):
    inflation_rate = get_inflation_rate(release_year)
    years_difference = 2024 - release_year   # because we are in 2024
    adjustment_factor = (1 + inflation_rate) ** years_difference
    adjusted_budget = budget * adjustment_factor
    return adjusted_budget

In [ ]:
#df['budget'] = df.apply(lambda row: adjust_for_inflation(row['budget'], row['release_year']), axis=1)
#df['revenue'] = df.apply(lambda row: adjust_for_inflation(row['revenue'], row['release_year']), axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1941 entries, 0 to 1994
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        1941 non-null   int64  
 1   budget                    1941 non-null   int64  
 2   genres                    1941 non-null   object 
 3   production_companies      1941 non-null   object 
 4   runtime                   1941 non-null   float64
 5   title                     1941 non-null   object 
 6   cast                      1941 non-null   object 
 7   revenue                   1941 non-null   int64  
 8   Action                    1941 non-null   int64  
 9   Adventure                 1941 non-null   int64  
 10  Animation                 1941 non-null   int64  
 11  Comedy                    1941 non-null   int64  
 12  Crime                     1941 non-null   int64  
 13  Documentary               1941 non-null   int64  
 14  Drama        

-----------

In [ ]:
features = ['actors','title','genres','runtime', 'runtime_ratio','lead_actor', 'supporting_actor_1', 'supporting_actor_2', 'director','writer','has_top_25_director','has_top_25_writer', 'has_top_25_company', 
            'in_english', 'produced_in_US','release_month','release_year','decade','season']

In [ ]:
log_features = ['revenue','budget','genre_rank','top_100_actor_count','num_production_countries', 'num_production_companies',
                'company_rank', 'actor_rank', 'director_rank', 'writer_rank']

In [ ]:
columns_to_include = list(set(features + log_features))

df = df[columns_to_include]

In [ ]:
columns_to_include

['produced_in_US',
 'runtime_ratio',
 'director',
 'supporting_actor_1',
 'runtime',
 'supporting_actor_2',
 'season',
 'revenue',
 'num_production_companies',
 'top_100_actor_count',
 'writer_rank',
 'num_production_countries',
 'actors',
 'decade',
 'budget',
 'actor_rank',
 'has_top_25_writer',
 'release_month',
 'lead_actor',
 'writer',
 'has_top_25_director',
 'in_english',
 'release_year',
 'has_top_25_company',
 'director_rank',
 'title',
 'company_rank',
 'genre_rank',
 'genres']

In [ ]:
df.to_csv('eda_movies.csv', index=False)

In [ ]:
df.drop(columns=['genres'],inplace=True)

In [ ]:
df1 = df[df.revenue >= 20000000]
df2 = df[df.revenue < 20000000]

In [ ]:
for feat in log_features:
    df1[feat] = np.log1p(df1[feat])
    df2[feat] = np.log1p(df2[feat])

/var/folders/pj/f8hslcw56411qpfnn2sxfqrm0000gn/T/ipykernel_83386/2989333006.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[feat] = np.log1p(df1[feat])
/var/folders/pj/f8hslcw56411qpfnn2sxfqrm0000gn/T/ipykernel_83386/2989333006.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[feat] = np.log1p(df2[feat])


In [ ]:
df1.to_csv('def_movies_high.csv', index=False)
df2.to_csv('def_movies_low.csv', index=False)

In [ ]:
df1.head()

,produced_in_US,runtime_ratio,director,supporting_actor_1,runtime,supporting_actor_2,season,revenue,num_production_companies,top_100_actor_count,writer_rank,num_production_countries,decade,budget,actor_rank,has_top_25_writer,release_month,lead_actor,writer,has_top_25_director,in_english,cast,release_year,has_top_25_company,director_rank,title,company_rank,genre_rank
1,1,0.056387,Garry Marshall,Julie Andrews,113.0,H√©ctor Elizondo,Summer,18.370959,0.693147,0.0,6.104793,0.693147,2000s,17.504390,5.958941,0,August,Anne Hathaway,Shonda Rhimes,0,1,"[Anne Hathaway, Julie Andrews, H√©ctor Elizond...",2004,1,5.068904,The Princess Diaries 2: Royal Engagement,3.178054,1.749200
5,1,0.045726,Ole Bornedal,Natasha Calis,92.0,Madison Davenport,Summer,18.263396,1.098612,0.0,6.025866,1.098612,2010s,16.454568,6.728629,0,August,Jeffrey Dean Morgan,Juliet Snowden,0,1,"[Jeffrey Dean Morgan, Natasha Calis, Madison D...",2012,0,5.755742,The Possession,5.323010,2.140066
7,1,0.060223,John G. Avildsen,Talia Shire,119.0,Burt Young,Autumn,18.579692,0.693147,0.0,5.123964,0.693147,1970s,13.815512,6.234411,0,November,Sylvester Stallone,Sylvester Stallone,0,1,"[Sylvester Stallone, Talia Shire, Burt Young, ...",1976,0,5.411646,Rocky,5.176150,1.098612
8,1,0.061031,Sam Mendes,Annette Bening,122.0,Thora Birch,Autumn,19.691274,1.098612,0.0,4.143135,0.693147,1990s,16.523561,6.327580,0,September,Kevin Spacey,Alan Ball,1,1,"[Kevin Spacey, Annette Bening, Thora Birch, We...",1999,0,3.891820,American Beauty,5.072044,1.098612
9,1,0.058853,F. Gary Gray,Uma Thurman,118.0,Vince Vaughn,Spring,18.371765,1.609438,0.0,6.580639,0.693147,2000s,17.785802,6.268338,0,March,John Travolta,Elmore Leonard,0,1,"[John Travolta, Uma Thurman, Vince Vaughn, Ced...",2005,0,5.749393,Be Cool,5.273000,1.945910


In [ ]:
df2.head()

,produced_in_US,runtime_ratio,director,supporting_actor_1,runtime,supporting_actor_2,season,revenue,num_production_companies,top_100_actor_count,writer_rank,num_production_countries,decade,budget,actor_rank,has_top_25_writer,release_month,lead_actor,writer,has_top_25_director,in_english,cast,release_year,has_top_25_company,director_rank,title,company_rank,genre_rank
0,1,0.046154,Steve Pink,Craig Robinson,93.0,Clark Duke,Winter,16.326300,1.386294,0.0,6.923629,0.693147,2010s,16.454568,6.616333,0,February,Rob Corddry,Josh Heald,0,1,"[Rob Corddry, Craig Robinson, Clark Duke, Adam...",2015,0,6.376727,Hot Tub Time Machine 2,4.912655,1.386294
2,1,0.052135,Damien Chazelle,J.K. Simmons,105.0,Melissa Benoist,Autumn,16.387512,1.386294,0.0,6.447306,0.693147,2010s,15.009433,6.601502,0,October,Miles Teller,Damien Chazelle,0,1,"[Miles Teller, J.K. Simmons, Melissa Benoist, ...",2014,0,6.717805,Whiplash,5.378360,1.098612
3,0,0.060636,Sujoy Ghosh,Nawazuddin Siddiqui,122.0,Parambrata Chatterjee,Spring,16.588099,0.000000,0.0,6.857514,0.693147,2010s,13.997833,6.728629,0,March,Vidya Balan,Sujoy Ghosh,0,0,"[Vidya Balan, Nawazuddin Siddiqui, Parambrata ...",2012,0,6.654153,Kahaani,5.568345,1.386294
4,0,0.041772,Hal Sutherland,Tom Bosley,83.0,Rickie Lee Jones,Summer,14.997740,0.000000,0.0,7.156956,0.000000,1980s,15.894952,6.478894,0,August,Scott Grimes,Collodi,0,1,"[Scott Grimes, Tom Bosley, Rickie Lee Jones, J...",1987,0,6.972606,Pinocchio and the Emperor of the Night,5.568345,1.897120
6,1,0.045432,Christopher Guest,Christopher Guest,91.0,John Michael Higgins,Spring,16.746717,0.693147,0.0,6.795706,0.693147,2000s,15.607270,6.200103,0,April,Bob Balaban,Christopher Guest,0,1,"[Bob Balaban, Christopher Guest, John Michael ...",2003,0,6.583409,A Mighty Wind,5.231109,2.302585
